# HDR gathering
Like basic gather, but with multiple pictures taken at different attenuations.

## Setup
Run `python f_camera_photonics\tcp_link.py` on the lab computer. Only one thing can use the analog port at a time, so make sure it is grabbing from the digital only.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import cv2
import sys, os
import numpy as np

sys.path.append('..')
import f_camera_photonics
from f_camera_photonics.component_capture import single_shot
from f_camera_photonics import cvshow
from f_camera_photonics.peak_finder import pick_ports, f_camera_photonics, main, get_all_config
from f_camera_photonics.tcp_link import remote_call, capture, attenuate, unpack_image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
remote = True
datadir = '../measurement_data/chicken01-cutbacks'
sample_name = 'foo'  # change this every device

## Preview valid box
You can do this with the lamp on, but make sure to turn them off afterward.

You will be given a preview of the valid box

In [7]:
if remote:
    img = unpack_image(remote_call(capture, avgcnt=1))
else:
    img = single_shot()
    
cfg=get_all_config()
if cfg.valid_box is not None:
    x, y, w, h = cfg.valid_box
    cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 2)
    
filename = os.path.join(datadir, sample_name + '.tif')
cv2.imwrite(filename, img)

cvshow(img)

/Users/ant12/Documents/git-projects/experiment-code/camera_photonics/f_camera_photonics/config.ini
Press any key to close the display window


### If you are satisfied with the valid box, move on

## Find the port box spec on the saved image
You will be given a preview of the port boxes

In [8]:
main(filename=filename, box_spec=None, max_box_width=10)

/Users/ant12/Documents/git-projects/experiment-code/camera_photonics/f_camera_photonics/config.ini


/Users/ant12/Documents/git-projects/experiment-code/camera_photonics/f_camera_photonics/peak_finder.py:311: RuntimeWarning: divide by zero encountered in log10
  img_8bit = np.log10(img_8bit)


(#1, P: 1.00)
(#2, P: 0.63)
Check if it is correct
Press any key to close the display window
Saving to foo.json in ../measurement_data/chicken01-cutbacks


{'Total Power': [161691.0, 102609.0],
 'Normalized Power': [1.0, 0.6345993283485166],
 'x': [298.0, 356.0],
 'y': [261.0, 264.0],
 'box_width': [10.0, 10.0]}

### If you are satisfied with port boxes, move on

## HDR image taking